# COSTO UNITARIO PROMEDIO

### OBTENCION COSTO DEL PRODUCTO

In [1]:
import xmlrpc.client
import pandas as pd

username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# 1. Obtener las órdenes de producción ya cerradas (hechas por manufactura)
ordenes_produccion = models.execute_kw(
    db, uid, password,
    'mrp.production', 'search_read',
    [[('state', '=', 'done')]],   # Solo cerradas
    {'fields': ['id', 'product_id', 'product_qty']}
)

# 2. Extraer los productos únicos de esas órdenes
ids_productos_manu = list({op['product_id'][0] for op in ordenes_produccion if op.get('product_id')})

# 3. Leer los productos manufacturados que correspondan a DONSSON
productos_manu = models.execute_kw(
    db, uid, password,
    'product.product', 'read',
    [ids_productos_manu],
    {'fields': ['id', 'name', 'standard_price', 'catalogo_codigo_fab', 'qty_available']}
)

# 4. Filtrar solo marca DONSSON
productos_donsson_manu = [p for p in productos_manu if p.get('catalogo_codigo_fab') == 'DONSSON']



productos_manu = pd.DataFrame(productos_manu) # productos con costo entre los manufacturados


In [2]:
productos_manu = productos_manu.rename(columns={
    'name': 'producto',
    'standard_price': 'costo_estandar_promedio',
    'qty_available': 'cantidad_disponible'
})

pd.set_option("display.max_colwidth", None)
productos_manu = productos_manu[["producto","costo_estandar_promedio","cantidad_disponible"]]

#### Buscador para casos especiales productos

In [3]:
productos_manu[productos_manu["producto"].str.contains("DA2919", case=False, na=False)]

,producto,costo_estandar_promedio,cantidad_disponible
291,DA2919 FILTROAIRE CHEV. LV150,45119.8,137.0


### OBTENCION COSTO DE ORDEN DE PRODUCCION

In [4]:
# 1. Obtener las órdenes de producción ya cerradas (hechas por manufactura)
ordenes_produccion = models.execute_kw(
    db, uid, password,
    'mrp.production', 'search_read',
    [[('state', '=', 'done')]],   # Solo cerradas
    {'fields': ['id','name'#'date_start'
                , 'product_id', 'product_qty','valor_unitario_mrp','date_finished']}
)



ops = pd.DataFrame(ordenes_produccion) # todas las OP cerradas

In [5]:
ops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11940 entries, 0 to 11939
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   valor_unitario_mrp  11940 non-null  float64
 1   product_id          11940 non-null  object 
 2   date_finished       11940 non-null  object 
 3   product_qty         11940 non-null  float64
 4   id                  11940 non-null  int64  
 5   name                11940 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 559.8+ KB


In [6]:
ops.head()

,valor_unitario_mrp,product_id,date_finished,product_qty,id,name
0,7481.39,"[17320, [DAB02995025] DA2995 FILTRO AIRE TOYOTA, MAZDA,FORD (025 DA2995)]",2018-03-28 13:00:43,1001.0,2388,O02445
1,24442.01,"[50106, [DAB08255025] DA8255 FILTRO AIRE - HITACHI - MAHINDRA (025 DA8255)]",2025-08-28 13:22:03,50.0,13365,MO13094
2,45837.31,"[42199, [DAR08194025] DA8194 FILTRO AIRE GENERADORES CUMMINS LTD C66-D5 (025 DA8194)]",2025-08-29 11:42:09,40.0,13364,MO13093
3,33019.38,"[17332, [DAR03067025] DA3067 FILTRO AIRE CASE, CAT, JOHN DEERE (025 DA3067)]",2025-08-29 11:43:41,30.0,13363,MO13092
4,27854.59,"[17054, [DAB02567025] DA2567 FILTRO AIRE 1_ DAEWOO MONTACARGAS (025 DA2567)]",2025-08-28 13:22:53,10.0,13362,MO13091


In [7]:
ops = ops.rename(columns={

    "name" :"nombre_op",
    "product_qty":"unidades_producidas",
    "valor_unitario_mrp":"costo_unitario_op",
    "date_finished":"fecha_cierre_op"
})


In [8]:
import re

# 1) Quedarnos con el nombre del producto dentro de product_id
#    (Odoo devuelve [id, "nombre"])
ops['product_name_raw'] = ops['product_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

# 2) Quitar el código entre corchetes "[...]" + espacios iniciales
# 3) Quitar todo lo que esté dentro de paréntesis "(...)" + espacios
ops['producto'] = (
    ops['product_name_raw']
      .astype(str)
      .str.replace(r'^\s*\[.*?\]\s*', '', regex=True)   # elimina "[CODE] "
      .str.replace(r'\(.*?\)', '', regex=True)          # elimina "(...)"
      .str.strip()
)

# (opcional) normalizar espacios internos
ops['producto'] = ops['producto'].str.replace(r'\s+', ' ', regex=True)




#### Buscador casos especiales por op

In [9]:
ops = ops[["producto","unidades_producidas","nombre_op","costo_unitario_op","fecha_cierre_op"]]
pd.set_option("display.max_colwidth", None)

ops[ops["producto"].str.contains("DA2919", case=False, na=False)]



,producto,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op
11,DA2919 FILTROAIRE CHEV. LV150,80.0,MO13055,44053.16,2025-08-28 13:23:34
339,DA2919 FILTROAIRE CHEV. LV150,60.0,MO12579,46616.85,2025-06-19 12:51:16
1078,DA2919 FILTROAIRE CHEV. LV150,100.0,MO11793,44671.96,2025-01-24 07:09:35
2388,DA2919 FILTROAIRE CHEV. LV150,200.0,MO10406,39022.32,2024-04-15 09:29:39
3416,DA2919 FILTROAIRE CHEV. LV150,200.0,MO09295,42877.23,2023-07-31 12:49:42
3969,DA2919 FILTROAIRE CHEV. LV150,150.0,MO08687,49438.50,2023-03-10 15:20:47
4354,DA2919 FILTROAIRE CHEV. LV150,150.0,MO08255,49528.94,2022-11-29 08:33:38
4705,DA2919 FILTROAIRE CHEV. LV150,100.0,MO07866,46465.06,2022-07-30 12:51:04
4926,DA2919 FILTROAIRE CHEV. LV150,50.0,MO07628,46831.95,2022-05-17 16:30:57
5056,DA2919 FILTROAIRE CHEV. LV150,50.0,MO07491,47031.55,2022-03-30 16:09:35


In [10]:
# Listar todos los campos posibles de órdenes de producción
campos = models.execute_kw(
    db, uid, password,
    'mrp.production', 'fields_get',
    []
)

# Mostrar algunos
for campo, info in list(campos.items())[:50]:
    print(campo, "→", info)


origin → {'sortable': True, 'help': 'Reference of the document that generated this production order request.', 'searchable': True, 'required': False, 'manual': False, 'readonly': True, 'states': {'draft': [['readonly', False]]}, 'depends': [], 'groups': False, 'company_dependent': False, 'translate': False, 'type': 'char', 'store': True, 'string': 'Source Document'}
message_follower_ids → {'sortable': False, 'string': 'Followers', 'searchable': True, 'required': False, 'manual': False, 'readonly': False, 'depends': [], 'relation': 'res.partner', 'groups': False, 'company_dependent': False, 'type': 'many2many', 'store': False}
tanda_id → {'sortable': True, 'string': 'Tanda', 'searchable': True, 'required': False, 'manual': False, 'readonly': False, 'depends': [], 'relation': 'mrp.production.tandas', 'groups': False, 'company_dependent': False, 'type': 'many2one', 'store': True}
allow_reorder → {'sortable': True, 'help': 'Check this to be able to move independently all production orders,

### Corrector matches merge

### OPS DE CADA PRODUCTO

In [11]:
from fuzzywuzzy import process

# 1. Merge exacto
df_unido = productos_manu.merge(ops, on="producto", how="left", indicator=True)

# 2. Productos que no hicieron match exacto
no_match = df_unido[df_unido["_merge"] == "left_only"]["producto"].unique()

# 3. Lista de productos en OP
productos_ops = ops["producto"].dropna().unique()

# 4. Crear diccionario de equivalencias con fuzzy
mapeo = {}
for prod in no_match:
    mejor_match, score = process.extractOne(prod, productos_ops)
    if score >= 85:   # solo aceptamos matches muy altos
        mapeo[prod] = mejor_match

# 5. Reemplazar en productos_manu
productos_manu["producto_corr"] = productos_manu["producto"].replace(mapeo)

# 6. Nuevo merge usando los nombres corregidos
df_unido_corr = productos_manu.merge(ops, left_on="producto_corr", right_on="producto", how="left", indicator=True)



/home/donsson/Produccion/.venv/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [12]:
error = df_unido_corr[df_unido_corr["_merge"]=="left_only"]
error.head(10)

,producto_x,costo_estandar_promedio,cantidad_disponible,producto_corr,producto_y,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op,_merge


### COSTO DE CADA OP

In [13]:
from fuzzywuzzy import process

# ==============================
# 1) Calcular costo promedio real de productos_manu
# ==============================
def calcular_costo_promedio(grupo):
    if grupo["cantidad_disponible"].sum() == 0:
        return None  # evitar división por 0
    return (grupo["costo_estandar_promedio"] * grupo["cantidad_disponible"]).sum() / grupo["cantidad_disponible"].sum()

costo_promedio = (
    productos_manu.groupby("producto")
    .apply(calcular_costo_promedio)
    .reset_index(name="costo_promedio")
)

# ==============================
# 2) Merge exacto primero (ops vs productos_manu)
# ==============================
df_ops_unido = ops.merge(costo_promedio, on="producto", how="left", indicator=True)

# ==============================
# 3) Detectar OPs sin match
# ==============================
no_match_ops = df_ops_unido[df_ops_unido["_merge"] == "left_only"]["producto"].unique()

# ==============================
# 4) Lista de productos válidos (de productos_manu)
# ==============================
productos_manu_lista = costo_promedio["producto"].dropna().unique()

# ==============================
# 5) Crear diccionario de equivalencias fuzzy
# ==============================
mapeo_ops = {}
for prod in no_match_ops:
    mejor_match, score = process.extractOne(prod, productos_manu_lista)
    if score >= 85:
        mapeo_ops[prod] = mejor_match

# ==============================
# 6) Corregir los nombres en ops
# ==============================
ops["producto_corr"] = ops["producto"].replace(mapeo_ops)

# ==============================
# 7) Merge final (con fuzzy corregido)
# ==============================
ops_con_costo = ops.merge(
    costo_promedio, left_on="producto_corr", right_on="producto", how="left"
)

# ==============================
# 8) Limpiar columnas duplicadas
# ==============================

cols_a_borrar = [c for c in ["producto_x", "producto_y"] if c in ops_con_costo.columns]
ops_con_costo = ops_con_costo.drop(columns=cols_a_borrar)

# Renombrar la corregida como "producto"
if "producto_corr" in ops_con_costo.columns:
    ops_con_costo = ops_con_costo.rename(columns={"producto_corr": "producto"})


/tmp/ipykernel_91273/787867308.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calcular_costo_promedio)


In [14]:
ops_con_costo.head()

,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op,producto,costo_promedio
0,1001.0,O02445,7481.39,2018-03-28 13:00:43,"DA2995 FILTRO AIRE TOYOTA, MAZDA,FORD",16351.45
1,50.0,MO13094,24442.01,2025-08-28 13:22:03,DA8255 FILTRO AIRE - HITACHI - MAHINDRA,23277.93
2,40.0,MO13093,45837.31,2025-08-29 11:42:09,DA8194 FILTRO AIRE GENERADORES CUMMINS LTD C66-D5,45837.35
3,30.0,MO13092,33019.38,2025-08-29 11:43:41,"DA3067 FILTRO AIRE CASE, CAT, JOHN DEERE",32759.09
4,10.0,MO13091,27854.59,2025-08-28 13:22:53,DA2567 FILTRO AIRE 1_ DAEWOO MONTACARGAS,27854.62


In [15]:
ops_con_costo['diferencia'] = ops_con_costo['costo_unitario_op'] - ops_con_costo['costo_promedio']

ops_con_costo["%diferencia"] = (ops_con_costo["diferencia"] / ops_con_costo["costo_promedio"])

ops_con_costo["%_diferencia_abs"] = (ops_con_costo["diferencia"].abs() / ops_con_costo['costo_promedio'])


In [16]:
ops_con_costo.to_excel("ops_relacionadas.xlsx")

In [17]:
ops_con_costo[ops_con_costo["costo_promedio"]==0]  # Comprobamos que no quedo ninguna OP sin el costo promedio asignado

,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op,producto,costo_promedio,diferencia,%diferencia,%_diferencia_abs


### Df unido y corregido

In [18]:
df_unido_corr.head(10)

,producto_x,costo_estandar_promedio,cantidad_disponible,producto_corr,producto_y,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op,_merge
0,"DA9134 FILTRO AIRE INGERSOLL RAND, DOOSAN",8137.04,0.0,"DA9134 FILTRO AIRE INGERSOLL RAND, DOOSAN","DA9134 FILTRO AIRE INGERSOLL RAND, DOOSAN",30.0,MO02400,6773.24,2018-04-12 07:42:14,both
1,"DA9134 FILTRO AIRE INGERSOLL RAND, DOOSAN",8137.04,0.0,"DA9134 FILTRO AIRE INGERSOLL RAND, DOOSAN","DA9134 FILTRO AIRE INGERSOLL RAND, DOOSAN",21.0,MO01061,2100.05,2017-03-17 07:39:25,both
2,DA4272 FILTRO AIRE 2_ TEREX,35169.06,23.0,DA4272 FILTRO AIRE 2_ TEREX,DA4272 FILTRO AIRE 2_ TEREX,20.0,MO06420,28054.50,2021-07-03 11:37:51,both
3,DA6003 FILTRO AIRE EQUIPO ESTACIONARIO WAUKESHA,85658.81,0.0,DA6003 FILTRO AIRE EQUIPO ESTACIONARIO WAUKESHA,DA6003 FILTRO AIRE EQUIPO ESTACIONARIO WAUKESHA,1.0,MO05090,85659.00,2020-05-12 13:06:27,both
4,"DA8109 FILTRO AIRE 1º JCB , NEW HOLLAND",20156.95,31.0,"DA8109 FILTRO AIRE 1º JCB , NEW HOLLAND","DA8109 FILTRO AIRE 1º JCB , NEW HOLLAND",50.0,MO12719,26525.09,2025-08-25 11:50:19,both
5,"DA8109 FILTRO AIRE 1º JCB , NEW HOLLAND",20156.95,31.0,"DA8109 FILTRO AIRE 1º JCB , NEW HOLLAND","DA8109 FILTRO AIRE 1º JCB , NEW HOLLAND",15.0,MO11818,51928.82,2025-02-27 09:47:01,both
6,FLANCHE DA4407,11564.69,0.0,FLANCHE DA4407,FLANCHE DA4407,40.0,MO10244,9700.56,2024-03-02 11:35:14,both
7,FLANCHE DA4407,11564.69,0.0,FLANCHE DA4407,FLANCHE DA4407,40.0,MO09934,13428.78,2024-01-12 12:20:34,both
8,FLANCHE DA4407,11564.69,0.0,FLANCHE DA4407,FLANCHE DA4407,200.0,MO06770,3778.96,2021-10-09 08:01:34,both
9,FLANCHE DA4407,11564.69,0.0,FLANCHE DA4407,FLANCHE DA4407,100.0,MO06431,3853.06,2021-07-22 10:06:17,both


In [19]:
# Crear una columna final estandarizada
df_unido_corr["producto"] = df_unido_corr["producto_corr"].fillna(df_unido_corr["producto_x"])

# Eliminar duplicadas
df_limpio = df_unido_corr.drop(columns=["producto_x", "producto_corr", "producto_y"])

df_limpio = df_limpio.sort_values(["producto", "fecha_cierre_op"], ascending=[True, False])

In [20]:
df_limpio = df_limpio[~ (df_limpio['cantidad_disponible']==0)]  #Retirar ruido , productos sin unidades disponibles

In [21]:
# ===========================
# 1) Función costo real
# ===========================
def calcular_costo_real(grupo):
    # ordenar las OP por fecha (descendente: más recientes primero)
    grupo = grupo.sort_values('fecha_cierre_op', ascending=False)

    cant_disp = grupo['cantidad_disponible'].iloc[0]
    total_tomado = 0
    costo_total = 0

    for _, row in grupo.iterrows():
        if total_tomado >= cant_disp:
            break

        # ignorar OP con costo nulo o cero
        if pd.isna(row['costo_unitario_op']) or row['costo_unitario_op'] == 0:
            continue

        # Tomar lo que falta o lo que tiene esta OP
        tomar = min(row['unidades_producidas'], cant_disp - total_tomado)

        costo_total += tomar * row['costo_unitario_op']
        total_tomado += tomar

    if total_tomado == 0:
        return None

    return costo_total / total_tomado


# ===========================
# 2) Costo real por producto
# ===========================
costo_real = (
    df_limpio
    .groupby('producto')
    .apply(calcular_costo_real)
    .reset_index()
)
costo_real.columns = ['producto', 'costo_real']


# ===========================
# 3) Última fecha de producción por producto
# ===========================
ultima_fecha = (
    df_limpio.groupby('producto')['fecha_cierre_op']
    .max()
    .reset_index()
)
ultima_fecha.columns = ['producto', 'ultima_fecha_produccion']


# ===========================
# 4) Construcción resumen
# ===========================
resumen = (
    df_limpio[['producto', 'costo_estandar_promedio', 'cantidad_disponible']]
    .drop_duplicates()
    .merge(costo_real, on='producto', how='left')
    .merge(ultima_fecha, on='producto', how='left')   # <-- se agrega la fecha
)

# Diferencia entre costo real y costo estándar
resumen['diferencia'] = resumen['costo_real'] - resumen['costo_estandar_promedio']

resumen["%diferencia"] = (resumen["diferencia"] / resumen["costo_estandar_promedio"])

resumen["%_diferencia_abs"] = (resumen["diferencia"].abs() / resumen["costo_estandar_promedio"])


/tmp/ipykernel_91273/3487751595.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calcular_costo_real)


In [22]:
resumen[resumen["costo_real"].isna()] # es un producto que tiene existencias y tiene una op pero en ella no cargaron costo unitario ejecutado fue en 2017

,producto,costo_estandar_promedio,cantidad_disponible,costo_real,ultima_fecha_produccion,diferencia,%diferencia,%_diferencia_abs
447,DA8021 FILTRO AIRE ASPIRADORAS INDUSTRIALES RIDGID,4856.01,7.0,NaN,2017-08-22 15:40:08,NaN,NaN,NaN


In [23]:
resumen[resumen["producto"].str.contains("DA2919", case=False, na=False)]


,producto,costo_estandar_promedio,cantidad_disponible,costo_real,ultima_fecha_produccion,diferencia,%diferencia,%_diferencia_abs
199,DA2919 FILTROAIRE CHEV. LV150,79500.58,141.0,44053.16,2025-08-28 13:23:34,-35447.42,-0.445876,0.445876
200,DA2919 FILTROAIRE CHEV. LV150,45119.80,137.0,44053.16,2025-08-28 13:23:34,-1066.64,-0.023640,0.023640


In [24]:
resumen.to_excel("monitoreo_costos.xlsx")

In [ ]:
import xmlrpc.client
import pandas as pd

username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# 1. Obtener las órdenes de producción ya cerradas (hechas por manufactura)
precio_estandar_historico = models.execute_kw(
    db, uid, password,
    'stock.change.standard.price', 'search_read',
    [[]],   # Todo
    {'fields': ['create_date', 'product_id', 'product_qty']}
)



# EQUIVALENCIAS MARCAS

In [6]:
import xmlrpc.client
import pandas as pd

username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

campos = models.execute_kw(
    db, uid, password,
    'product.template', 'fields_get',
    []
)

for nombre, info in campos.items():
    if info.get('type') in ('one2many', 'many2one', 'many2many'):
        print(nombre, "->", info.get('relation'))




write_uid -> res.users
property_account_income -> account.account
website_style_ids -> product.style
product_rate -> website.product.rate
property_stock_account_output -> account.account
catalogo_valvula_alivio -> catalogo.product.valvula_alivio
company_id -> res.company
public_categ_ids -> product.public.category
pricelist_id -> product.pricelist
property_stock_account_input -> account.account
bom_ids -> mrp.bom
classprod_id -> cs.clasprod
catalogo_material_filtrante -> catalogo.product.material_filtrante
alternative_product_ids -> product.template
seller_id -> res.partner
packaging_ids -> product.packaging
pos_categ_id -> pos.category
seller_ids -> product.supplierinfo
website_message_ids -> mail.message
catalogo_marca -> catalogo.marcas
accessory_product_ids -> product.product
message_follower_ids -> res.partner
attribute_line_ids -> product.attribute.line
asset_category_id -> account.asset.category
images -> product.image
catalogo_rosca -> catalogo.product.rosca
catalogo_forma -> c

In [15]:
import xmlrpc.client
import pandas as pd

username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# 1) Productos básicos
productos = models.execute_kw(
    db, uid, password,
    'product.template', 'search_read',
    [[]],
    {'fields': ['id', 'name', 'default_code']}
)
df_prod = pd.DataFrame(productos).rename(columns={
    'id': 'product_id',
    'name': 'producto_nombre',
    'default_code': 'producto_codigo'
})

# 2) Equivalencias
equivalencias = models.execute_kw(
    db, uid, password,
    'catalogo.referencia.fabricantes', 'search_read',
    [[]],
    {'fields': ['id', 'name', 'tipo_marca', 'productos_referencia']}
)
df_eq = pd.DataFrame(equivalencias)

# 3) Expandir la lista de productos_referencia
df_eq = df_eq.explode('productos_referencia')
df_eq = df_eq.rename(columns={'name': 'equivalencia_ref'})

# 4) Limpiar tipo_marca (de lista [id, nombre] → solo nombre)
df_eq['marca'] = df_eq['tipo_marca'].apply(lambda x: x[1] if isinstance(x, list) else None)
df_eq['productos_referencia'] = df_eq['productos_referencia'].astype('Int64')

# 5) Merge con productos
df_final = df_eq.merge(df_prod, left_on='productos_referencia', right_on='product_id', how='left')
df_final = df_final.dropna()

# 6) Exportar a Excel
df_final[['producto_codigo', 'producto_nombre', 'equivalencia_ref', 'marca']].to_excel("equivalencias.xlsx", index=False)
print("✅ Archivo 'equivalencias.xlsx' generado correctamente")


✅ Archivo 'equivalencias.xlsx' generado correctamente
